In [1]:
#### Written and Copyright by Mohit Agarwal
#### Georgia Institute of Technology
#### Email: magarwal37@gatech.edu

### Code to decode Error-Potentials for the data recording on any game
### Data should be present in the format of SXX-MMDD-XX.csv, _labels, and _metadata
### 16 Electrode Channels: desribed in metadata file e.g. ['Pz', 'F4', 'C4', 'P4', 'O2', 'F8', 'T4', 'Cz', 'Fz', 'F3', 'C3', 'P3', 'O1', 'F7', 'T3', 'Fpz']

### If want to classify data for every subject individually - put separate folder for every subject
### If training and testing on different files, the folder structure should be similar in both train and test

In [2]:
##Importing Libraries

import numpy as np
import pandas as pd
import bisect
import os
import math
from scipy.signal import *
from pylab import *
from random import shuffle
from collections import Counter
from sklearn.preprocessing import Normalizer
from sklearn.linear_model import ElasticNet, LogisticRegression, SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix
from pyriemann.utils.viz import plot_confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold, train_test_split
from sklearn.calibration import CalibratedClassifierCV
from random import shuffle
from sklearn import svm
import matplotlib
import matplotlib.pyplot as plt
import pyriemann
from tabulate import tabulate

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE

/home/mohit/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/home/mohit/anaconda3/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [3]:
## Fixed Parameters of the Algorithm
data_dir = 'data'
train_dir = 'train'
test_dir = 'test'
channels = ['Pz', 'F4', 'C4', 'P4', 'O2', 'F8', 'Fp2', 'Cz', 'Fz', 'F3', 'C3', 'P3', 'O1', 'F7', 'Fp1', 'Fpz']

stim_nonErrp = 1
stim_Errp = 0
th_1 = 75 # in uV : for adjacent spikes
th_2 = 150 # in uV : for min-max
freq = 125.0
time_delay = 0.0 # in seconds

In [4]:
## Parameters of the algorithm [to change]

# Misc
flag_test = False  # True if testing o.w. Cross-validate
plt_err_dist = False  # Plots Error Distribution in time if set True
selected_channels = [0,1,2,3,7,8,9,10,11,15] # If empty means all channels are selected, other option: [0,4,5]
balanced_sampling = "undersample" # options are "undersample", "oversample", "smote, "None"
sync_method = "event_date" # options are "default" and "event_date"

# Classification based
low_freq = 1.0
baseline_epoc = int(0.1*freq)
butter_filt_order = 4
cv_folds = 10
xdawn_filters = 4
algo = "ner" # options are mit,ner,custom

# Plot-based
plot_GAERP_allchan = False
plot_GAERP_onechan = False
plot_ERP_onechan = False
chan_toPlot = 4


In [5]:
# Setting Per-Processing and Classification parameters as per the chosen algorithm

if algo is "mit":
    mean_baseline = False
    mean_per_chan = False
    car_pre = False
    epoc_window = int(0.8*freq)
    high_freq = 40.0
    car_post = True
    class_model = "mit"
elif algo is "ner":
    mean_baseline = False
    mean_per_chan = True
    car_pre = False
    epoc_window = int(1.3*freq)
    high_freq = 40.0
    car_post = False
    class_model = "ner"
else:
    mean_baseline = True
    mean_per_chan = False
    car_pre = False#True
    epoc_window = int(0.8*freq)
    high_freq = 10.0
    car_post = False
    class_model = "ner"

total_channels = 16 if selected_channels is None else len(selected_channels) 
if selected_channels:
    channels = [channels[i] for i in selected_channels]


In [6]:
## Helper Functions

# function to convert missing values to 0
convert = lambda x: float(x.strip() or float('NaN'))

def convert_codes(x):
    if ':' in x:
        return x.split(':',1)[1]
    else:
        return (x.strip() or float('NaN'))

# function to convert stimulations
def to_byte(value, length):
    for x in range(length):
        yield value%256
        value//=256

# function to bandpass filter
def bandpass(sig,band,fs,butter_filt_order):
    B,A = butter(butter_filt_order, np.array(band)/(fs/2), btype='bandpass')
    return lfilter(B, A, sig, axis=0)

In [7]:
def loadData(DataFolder, label_file):
    raw_file = label_file.split('_')[0] + '.csv'
    raw_EEG = np.loadtxt(open(os.path.join(DataFolder,raw_file), "rb"), delimiter=",", skiprows=1, usecols=(0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17))
    stim = pd.read_csv(os.path.join(DataFolder,label_file))
    return raw_EEG, stim

def pre_process(raw_EEG):
    # Step 1: Subtract Mean per electrode
    sig = raw_EEG[:,1:]
    sig = sig[:,selected_channels] if selected_channels is not None else sig
    if mean_per_chan:
        sig_mean = np.mean(sig, axis=0)
        sig = sig - sig_mean
        
    # Step 2: Bandpass in the given frequency range
    sigF = bandpass(sig, [low_freq,high_freq], freq, butter_filt_order)
    
    # Step 3: Removing Average of all channels
    if car_pre:
        sig_mean = np.mean(sigF, axis=1)
        sigF = sigF - np.reshape(sig_mean, [len(sig_mean),1])*np.ones([1,total_channels])
    
    return sigF

def post_process(X):
    if car_post:
        temp =  np.repeat(np.mean(X,axis=2),X.shape[2],axis=1)
        temp = np.reshape(temp, X.shape)
        X = X - temp
    X = X.transpose((0,2,1))
    return X
    

# Arrange EEG data for training/testing
def arrange_data(EEG, sigF, stim):
    X = []
    Y = []
    for stim_id, stim_code in enumerate(stim['label']):
        if (not math.isnan(stim_code)) and stim_code!=33552 and stim_code!=33553 and stim_code!=33554:
            if stim_code==stim_nonErrp or stim_code==stim_Errp:
                if sync_method == "default":
                    time_instant = stim['time1'][stim_id] + time_delay
                else:
                    time_instant = stim['time2'][stim_id] + time_delay
                time_instant = time_instant
                idx = bisect.bisect(EEG[:,0],time_instant)
                X_temp = sigF[idx-1:idx+epoc_window,:]
                if mean_baseline:
                    X_mean = np.mean(sigF[idx-baseline_epoc:idx,:],0)
                    X_temp = X_temp - X_mean
                check = False
                # Removing nan values and corrupted data
                if np.isnan(X_temp).any():
                    check = True
                for i in range(total_channels):  
                    X_diff = [abs(t - s) for s, t in zip(X_temp[:,i], X_temp[1:,i])]
                    if max(X_diff)>th_1 or max(X_temp[:,i])-min(X_temp[:,i])>th_2:
                        check = True
                if not check:
                    X.append(X_temp)
                    Y.append(stim_code)
    return np.array(X), np.array(Y)

In [8]:
# Pre-processing Data from Train/Test Directory

def dir_list(folder_path):
    dir_list = [d for d in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, d))]
    dir_list.append('')
    return dir_list

data_dict = {}
train_folder = os.path.join(data_dir,train_dir)
test_folder = os.path.join(data_dir,test_dir)
list_of_dir = dir_list(train_folder)
if flag_test:
    assert list_of_dir==dir_list(test_folder), "Error: Test directory does not have same structure as train"
total_dirs = len(list_of_dir)
for dir_idx in range(total_dirs):
    paths = [["train", os.path.join(train_folder, list_of_dir[dir_idx])], ["test", os.path.join(test_folder, list_of_dir[dir_idx])]]
    if not flag_test:
        paths.pop(1)
    for items in paths:
        train_test = items[0]
        folder = items[1]
        list_of_files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f)) and '_labels' in f]
        print(list_of_files)
        for curr_file in list_of_files:
            
            # Step 1: Load Raw EEG data (with filtered stimulations)
            EEG, stim = loadData(folder, curr_file)
            
            # Step 2: Pre-process the raw EEG data before cutting in time-windows
            sig = pre_process(EEG)
            
            # Step 3: Arrange Data
            X_curr, Y_curr = arrange_data(EEG, sig, stim)
            
            # Making sure either the stimulations are present or raw data during stimulation is not discarded
            if Y_curr.shape[0] > 0:
                # Step 4: Post-process the Signals
                X_curr = post_process(X_curr)

                # Step 5: Arrange data in dictionary
                key = 'data_' + list_of_dir[dir_idx] + '$' + train_test
                if key in data_dict:
                    [X, Y] = data_dict[key]
                    X = np.concatenate((X,X_curr),axis=0)
                    Y = np.concatenate((Y,Y_curr),axis=0)
                    data_dict[key] = [X, Y]
    #                 print X.shape, Y.shape
                else:
                    data_dict[key] = [X_curr, Y_curr]
                
            
exps = set()
for key in data_dict.keys():
    exps.add(key.split("$")[0])

# elements in the 'exps' set holds the folder names
# [X_train, Y_train] = data_dict[exp+"$train"] loads the X_train and Y_train for training samples
# All samples in all files inside one folder are concatenated when storing in data_dict

['S04-0628-06_labels.csv', 'S04-0628-02_labels.csv', 'S04-0628-08_labels.csv', 'S04-0628-03_labels.csv', 'S04-0628-04_labels.csv', 'S04-0628-05_labels.csv', 'S04-0628-07_labels.csv', 'S04-0628-10_labels.csv', 'S04-0628-09_labels.csv', 'S04-0628-01_labels.csv']
['S01-0719-06_labels.csv', 'S01-0719-05_labels.csv', 'S01-0719-02_labels.csv', 'S01-0719-03_labels.csv', 'S01-0719-04_labels.csv', 'S01-0719-01_labels.csv']
['S15-0909-05_labels.csv', 'S15-0909-07_labels.csv', 'S15-0909-10_labels.csv', 'S15-0909-04_labels.csv', 'S15-0909-06_labels.csv', 'S15-0909-09_labels.csv', 'S15-0909-02_labels.csv', 'S15-0909-01_labels.csv', 'S15-0909-03_labels.csv', 'S15-0909-08_labels.csv']
['S06-0801-08_labels.csv', 'S06-0801-10_labels.csv', 'S06-0801-04_labels.csv', 'S06-0801-03_labels.csv', 'S06-0801-06_labels.csv', 'S06-0801-02_labels.csv', 'S06-0801-09_labels.csv', 'S06-0801-07_labels.csv', 'S06-0801-01_labels.csv', 'S06-0801-05_labels.csv']
['S03-0628-03_labels.csv', 'S03-0628-04_labels.csv', 'S03-06

In [9]:
# Different Classification Pipelines after pre-processing

def pipeline_custom(X_train, X_test, Y_train):
    
    filt_1 = pyriemann.estimation.XdawnCovariances(nfilter=xdawn_filters, estimator='lwf', xdawn_estimator='lwf')
    #filt_3 = pyriemann.tangentspace.TangentSpace(metric='logeuclid', tsupdate=False)
    #filt_5 = ElasticNet(l1_ratio=0.05, alpha=0.02, normalize=False) #KNeighborsClassifier(n_neighbors=5) #svm.SVC(kernel='linear', gamma='scale')
    filt_5 = pyriemann.classification.MDM(metric='riemann')
                                                           
    clf = make_pipeline(filt_1, filt_5)
    clf.fit(X_train, Y_train)
    train_scores = clf.predict(X_train)
                                                           
    # Testing Data
    test_scores = clf.predict(X_test)
    return train_scores, test_scores

def pipeline_ner(X_train, X_test, Y_train):
    
    filt_1 = pyriemann.estimation.XdawnCovariances(nfilter=xdawn_filters, applyfilters=False, estimator='lwf', xdawn_estimator='lwf')
    filt_2 = pyriemann.channelselection.ElectrodeSelection(nelec=8, metric='riemann')
    filt_3 = pyriemann.tangentspace.TangentSpace(metric='riemann', tsupdate=False) # 36-sized vector for nelec=8
    filt_4 = Normalizer(norm='l1')                                                       
    filt_5 = ElasticNet(l1_ratio=0.05, alpha=0.02, normalize=True)
#     filt_5 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(16, 4))
#     filt_5 = SGDClassifier(loss='squared_hinge', penalty='elasticnet', l1_ratio=0.5, alpha=0.002, max_iter=1000, tol=0.001)
#     filt_5 = 
                                                           
#     clf = make_pipeline(filt_1, filt_2, filt_3, filt_4)
#     clf.fit(X_train, Y_train)
#     train_features = clf.transform(X_train)
#     test_features = clf.transform(X_test)
    
#     clf_isotonic = CalibratedClassifierCV(filt_5, cv=2, method='isotonic')
#     clf_isotonic.fit(train_features, Y_train)
#     train_scores = clf_isotonic.predict_proba(train_features)[:, 1]
#     test_scores = clf_isotonic.predict_proba(test_features)[:, 1]
    
#     X_train = X_train[:,:8,:]
#     print "xtrain: ", X_train.shape
#     filt_x = pyriemann.spatialfilters.Xdawn(nfilter=4)
#     temp = filt_x.fit_transform(X_train, Y_train)
#     print "xdawn: ", temp.shape
# #     print filt_x.evokeds_.shape
    
#     filt_y = pyriemann.estimation.XdawnCovariances(nfilter=4, applyfilters=False)
#     temp2 = filt_y.fit_transform(X_train, Y_train)
#     print "xdawncov: ", temp2.shape
    
    clf = make_pipeline(filt_1, filt_2, filt_3, filt_4, filt_5)
    clf.fit(X_train, Y_train)
    train_scores = clf.predict(X_train)
                                                           
    # Testing Data
    test_scores = clf.predict(X_test)
    return train_scores, test_scores

def pipeline_mit(X_train, X_test, Y_train):
    
    ind_0 = np.argwhere(Y_train==0).flatten()
    ind_1 = np.argwhere(Y_train==1).flatten()
    X_train_0 = X_train[ind_0,:,:]
    X_train_1 = X_train[ind_1,:,:]
    mean_class_0 = np.mean(X_train_0, axis=0)
    mean_class_1 = np.mean(X_train_1, axis=0)
    
    filt_1 = pyriemann.estimation.XdawnCovariances(nfilter=xdawn_filters, estimator='lwf', xdawn_estimator='lwf')
    filt_2 = pyriemann.tangentspace.TangentSpace(metric='logeuclid', tsupdate=False)
    filt_3 = ElasticNet(l1_ratio=0.1, alpha=0.0002, normalize=False)
    X_fe_1 = X_train
    X_fe_1 = filt_1.fit_transform(X_fe_1, Y_train)
    X_fe_1 = filt_2.fit_transform(X_fe_1, Y_train)
    X_fe_2 = np.zeros([X_train.shape[0],16])
    channel_indices = range(0, total_channels) if selected_channels is None else selected_channels
    for sample_idx in range(0,X_train.shape[0]):
        for chan_idx in channel_indices:
            mean_corr_0 = np.corrcoef(mean_class_0[chan_idx,:],X_train[sample_idx,chan_idx,:])[0,1]
            mean_corr_1 = np.corrcoef(mean_class_1[chan_idx,:],X_train[sample_idx,chan_idx,:])[0,1]
            X_fe_2[sample_idx, chan_idx] = mean_corr_0 - mean_corr_1
    X_fe = np.concatenate((X_fe_1,X_fe_2),axis=1)
    filt_3.fit(X_fe, Y_train)
    train_scores = filt_3.predict(X_fe)
    
    # Testing Data
    Xtest_fe1 = X_test
    Xtest_fe1 = filt_1.transform(Xtest_fe1)
    Xtest_fe1 = filt_2.transform(Xtest_fe1)
    Xtest_fe2 = np.zeros([X_test.shape[0],16])
    for sample_idx in range(0,X_test.shape[0]):
        for chan_idx in range(0,16):
            mean_corr_0 = np.corrcoef(mean_class_0[chan_idx,:],X_test[sample_idx,chan_idx,:])[0,1]
            mean_corr_1 = np.corrcoef(mean_class_1[chan_idx,:],X_test[sample_idx,chan_idx,:])[0,1]
            Xtest_fe2[sample_idx, chan_idx] = mean_corr_0 - mean_corr_1
    Xtest_fe = np.concatenate((Xtest_fe1,Xtest_fe2),axis=1)
    test_scores = filt_3.predict(Xtest_fe)
    return train_scores, test_scores


In [10]:
def balance_dataset(X_train, Y_train):
    # For balanced sampling
    if balanced_sampling == "undersample":
        rusU = RandomUnderSampler(return_indices=True)
        X_new = np.reshape(X_train,[X_train.shape[0],X_train.shape[1]*X_train.shape[2]])
        X_resU, Y_resU, id_rusU = rusU.fit_resample(X_new, Y_train)
        X_new2 = np.reshape(X_resU,[X_resU.shape[0],X_train.shape[1],X_train.shape[2]])
        X_sim = X_new2
        Y_sim = Y_resU
    elif balanced_sampling == "oversample":
        rusO = RandomOverSampler(return_indices=True)
        X_new = np.reshape(X_train,[X_train.shape[0],X_train.shape[1]*X_train.shape[2]])
        X_resO, Y_resO, id_rusO = rusO.fit_resample(X_new, Y_train)
        X_new2 = np.reshape(X_resO,[X_resO.shape[0],X_train.shape[1],X_train.shape[2]])
        X_sim = X_new2
        Y_sim = Y_resO
    elif balanced_sampling == "smote":
        rusS = SMOTE()
        X_new = np.reshape(X_train,[X_train.shape[0],X_train.shape[1]*X_train.shape[2]])
        X_resS, Y_resS = rusS.fit_resample(X_new, Y_train)
        X_new2 = np.reshape(X_resS,[X_resS.shape[0],X_train.shape[1],X_train.shape[2]])
        X_sim = X_new2
        Y_sim = Y_resS
    else:
        X_sim = X_train
        Y_sim = Y_train
    return X_sim, Y_sim

In [11]:
# Acutal Prediction

def predict(X_train, X_test, Y_train, Y_test):
    train_preds = []
    test_preds = []
    if class_model=="mit":
        train_scores, test_scores = pipeline_mit(X_train, X_test, Y_train)
        thresholds = [x/100.0 for x in range(20,81,1)]
        acc_tr = []
        for th in thresholds:
            acc_tr.append([np.mean((train_scores > th)==Y_train),th])
        decision_th = max(acc_tr)[1]
        auc_score = roc_auc_score(Y_test, test_scores)
        train_preds =  train_scores > decision_th
        test_preds = test_scores > decision_th
    elif class_model=="ner":
        train_scores, test_scores = pipeline_ner(X_train, X_test, Y_train)
        thresholds = [x/100.0 for x in range(20,81,1)]
        acc_tr = []
        for th in thresholds:
            acc_tr.append([np.mean((train_scores > th)==Y_train),th])
        decision_th = max(acc_tr)[1]
        auc_score = []
        try:
            auc_score.append(roc_auc_score(Y_test, test_scores))
        except ValueError:
            pass
        train_preds =  train_scores > decision_th
        test_preds = test_scores > decision_th
    else:
        train_scores, test_scores = pipeline_custom(X_train, X_test, Y_train)
        thresholds = [x/100.0 for x in range(20,81,1)]
        acc_tr = []
        for th in thresholds:
            acc_tr.append([np.mean((train_scores > th)==Y_train),th])
        decision_th = max(acc_tr)[1]
        auc_score = roc_auc_score(Y_test, test_scores)
        train_preds =  train_scores > decision_th
        test_preds = test_scores > decision_th
    return confusion_matrix(Y_train, train_preds), confusion_matrix(Y_test, test_preds), test_preds, mean(auc_score)

In [12]:
# Cross Validation: Returns train and test confusion matrix for X and Y

def predict_CV(X, Y):    
    kfold = KFold(n_splits=cv_folds, shuffle=True)
    cmat_train = np.zeros([2,2])
    cmat_test = np.zeros([2,2])
    auc_test = 0
    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
      
        X_train, Y_train = balance_dataset(X_train, Y_train)
            
        # print "Training ErrPs: ", np.argwhere(Y_train==0).shape[0]*100.0/(X_train.shape[0]), "%" 
        # print "Validation ErrPs: ", np.argwhere(Y_test==0).shape[0]*100.0/(X_test.shape[0]), "%"
        cm_train, cm_test, _, auc_curr = predict(X_train, X_test, Y_train, Y_test)
        cmat_train = cmat_train + cm_train
        cmat_test = cmat_test + cm_test
        auc_test = auc_test + auc_curr

    return cmat_train, cmat_test, auc_test*1.0/cv_folds

In [13]:
# Computing Accuracies

result_dict = {}
cmat_train, cmat_test, auc_test = np.zeros([2,2]), np.zeros([2,2]), []

for exp_idx, exp in enumerate(exps):
    print("Processing.. "+str(exp_idx)+"/"+str(len(exps))+" :"+exp)
    Y_pred = None
    Y_test = None
    if flag_test:
        [X_train, Y_train] = data_dict[exp+"$train"]
        index_shuffle = range(X_train.shape[0])
        shuffle(index_shuffle)
        X_train = X_train[index_shuffle,:,:]
        Y_train = Y_train[index_shuffle]
        [X_test, Y_test] = data_dict[exp+"$test"]
        X_train, Y_train = balance_dataset(X_train, Y_train)
        cmat_train, cmat_test, Y_pred, auc_test = predict(X_train, X_test, Y_train, Y_test)
    else:
        cmat_train, cmat_test, auc_test = np.zeros([2,2]), np.zeros([2,2]), []
        num_sims = 10
        [X_train, Y_train] = data_dict[exp+"$train"]
        for ind in range(num_sims):
            a, b, c = predict_CV(X_train,Y_train)
            cmat_train = cmat_train + a
            cmat_test = cmat_test + b
            auc_test.append(c)
        cmat_train = cmat_train/num_sims
        cmat_test = cmat_test/num_sims
        auc_test = mean(auc_test)
    
    assert not exp in result_dict.keys()
    train_errp_split = (sum(cmat_train,1)*1.0/sum(cmat_train))[0]
    test_errp_split = (sum(cmat_test,1)*1.0/sum(cmat_test))[0]
    result_dict[exp] = [cmat_train, cmat_test, X_train.shape[0], train_errp_split, test_errp_split, Y_test, Y_pred, auc_test]
        

Processing.. 0/21 :data_S08-0807
Processing.. 1/21 :data_S07-0716


/home/mohit/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/mohit/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/mohit/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/mohit/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/mohit/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/mohit/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Processing.. 2/21 :data_S04-0628
Processing.. 3/21 :data_S06-0801
Processing.. 4/21 :data_S05-0628
Processing.. 5/21 :data_S16-0910
Processing.. 6/21 :data_S02-0717
Processing.. 7/21 :data_S07-0626
Processing.. 8/21 :data_S01-0719
Processing.. 9/21 :data_S01-0629
Processing.. 10/21 :data_S01-0724
Processing.. 11/21 :data_S07-0718
Processing.. 12/21 :data_S12-0803


/home/mohit/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/mohit/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Processing.. 13/21 :data_S07-0729
Processing.. 14/21 :data_S02-0716
Processing.. 15/21 :data_S09-0628
Processing.. 16/21 :data_S09-0627


/home/mohit/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/mohit/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/mohit/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/mohit/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/mohit/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/mohit/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Processing.. 17/21 :data_S15-0909
Processing.. 18/21 :data_S01-0717


/home/mohit/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/mohit/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/mohit/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/mohit/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Processing.. 19/21 :data_S02-0718
Processing.. 20/21 :data_S03-0628


In [14]:
print_results_1 = []
print_results_2 = []
for key in result_dict.keys():
    [cmat_train, cmat_test, num_samples, tr_sp, te_sp, Y_test, Y_pred, auc_test] = result_dict[key]
    train_acc = cmat_train.diagonal()*1.0/cmat_train.sum(axis=1)
    test_acc = cmat_test.diagonal()*1.0/cmat_test.sum(axis=1)
    test_prec = cmat_test[0,0]*1.0/(cmat_test[0,0] + cmat_test[1,0])
    f1score = 2*test_acc[0]*test_prec/(test_acc[0] + test_prec)
    print_results_1.append([key, mean(test_acc), test_prec, auc_test, f1score, test_acc[0],test_acc[1]])
    print_results_2.append([key,num_samples,tr_sp, te_sp, mean(train_acc),train_acc[0], train_acc[1]])
    
    # Plot error distribution in time
    if plt_err_dist and flag_test:
        decision = (Y_pred == Y_test)
        x = [ind_x for ind_x in range(len(Y_test))]
        x = np.array(x)
        correct_idx = np.argwhere(decision==True)
        incorrect_idx = np.argwhere(decision==False)
        plt.plot(x[correct_idx],Y_test[correct_idx],'b.',label='Correct')
        plt.plot(x[incorrect_idx],Y_test[incorrect_idx],'r.',label='Incorrect')
        plt.xlabel('timesteps')
        yticks(np.array([-5, 0, 1, 5]), ('','ErrP', 'Non-Errp',''))
        plt.legend()
        vert_line = 100
        while vert_line < len(Y_test):
            plt.axvline(x=vert_line)
            vert_line = vert_line + 100
        plt.title(key)
        plt.show()
    
print(tabulate(list(print_results_1), headers=['Name', 'Avg. Acc', 'Precision', 'AUC', 'F1 Score', 'ErrP Acc', 'Non_ErrP Acc']))
# print("")
print(tabulate(list(print_results_2), headers=['Name', '#Samples', 'Tr_split', 'Te_split', 'Train Avg. Acc', 'Train ErrP Acc', 'Train Non_ErrP Acc']))


Name             Avg. Acc    Precision         AUC    F1 Score    ErrP Acc    Non_ErrP Acc
-------------  ----------  -----------  ----------  ----------  ----------  --------------
data_S08-0807    0.580848     0.280284    0.613495    0.380985    0.594624        0.567073
data_S07-0716    0.855769     0.33558   nan           0.492095    0.922222        0.789316
data_S04-0628    0.684923     0.349456    0.754601    0.459825    0.672093        0.697753
data_S06-0801    0.728609     0.37814     0.82022     0.503389    0.752703        0.704516
data_S05-0628    0.719779     0.310897    0.823187    0.436445    0.732075        0.707483
data_S16-0910    0.760841     0.452276    0.844125    0.56462     0.75122         0.770462
data_S02-0717    0.659561     0.55425     0.709824    0.605133    0.666304        0.652817
data_S07-0626    0.858118     0.631285    0.941551    0.724691    0.850538        0.865698
data_S01-0719    0.808542     0.712207    0.876761    0.74756     0.786607        0.830476